# Part 1.6. Softmax Classification

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

## 1. Softmax
**부드럽게(soft) 최대값(max)을 뽑아준다**는 말처럼 입력값에 대해 일어날 확률을 구해준다. 이 때 입력값의 결과인 확률의 합은 1이다.

$$P(class = i) = \frac{e^i}{\sum e^i}$$


In [3]:
ft = torch.FloatTensor([1, 2, 3])
print(F.softmax(ft, dim=0))
print(F.softmax(ft, dim=0).sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


## 2. Cross Entropy
$y$는 one-hot-encoding된 값이고, $\hat{y}$는 encoding된 각각의 값이 나올 수 있는 확률을 말한다. 만약 확률이 0에 가깝다면 $\log{\hat y}$는 $-\infty$에 가까워지고 1에 가까워진다면 $0$에 가까워진다. 

이를 각각의 확률에 one-hot-encoding된 값을 곱한 다음 거기에 $-$를 곱하면 **y가 발생하지 않을 확률**을 구할 수 있다. 이 확률들을 다 더하여 평균을 내면 우리가 원하는 cost function이 된다.

$$L = \frac{1}{N} \sum -y\log{\hat y}$$



In [21]:
pred = torch.rand(3, 5, requires_grad=True)
y = torch.randint(5, (3,)).long()
y_one_hot = torch.zeros_like(pred).scatter(1, y.unsqueeze(1), 1)
cost = (y_one_hot * - torch.log(F.softmax(pred, dim=1))).sum(dim=1).mean()

print(pred)
print(y_one_hot)
print(cost)

tensor([[0.9891, 0.1500, 0.6211, 0.1303, 0.9269],
        [0.3060, 0.8012, 0.5149, 0.4611, 0.4840],
        [0.5850, 0.7357, 0.5802, 0.6525, 0.0502]], requires_grad=True)
tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])
tensor(1.8316, grad_fn=<MeanBackward0>)


In [22]:
# log_softmax와 nll_lose 함수로 계산 가능
print(F.nll_loss(F.log_softmax(pred, dim=1), y))

# cross_entropy 함수로 계산 가능
print(F.cross_entropy(pred, y))

tensor(1.8316, grad_fn=<NllLossBackward>)
tensor(1.8316, grad_fn=<NllLossBackward>)


## 3. Training with Cross Entrophy Loss

In [0]:
# 데이터셋
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [0]:
# 모델 정의
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)  # 입력의 크기 4, 출력의 크기 3 (0, 1, 2)

    def forward(self, x):
        return self.linear(x)          # (m, 3)

In [0]:
model = SoftmaxClassifierModel()

In [0]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [34]:
# 반복 학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # epoch 100번 마다 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 2.077572
Epoch  100/1000 Cost: 0.703077
Epoch  200/1000 Cost: 0.625377
Epoch  300/1000 Cost: 0.570115
Epoch  400/1000 Cost: 0.520952
Epoch  500/1000 Cost: 0.474237
Epoch  600/1000 Cost: 0.428575
Epoch  700/1000 Cost: 0.383289
Epoch  800/1000 Cost: 0.338086
Epoch  900/1000 Cost: 0.293336
Epoch 1000/1000 Cost: 0.253128


## 4. Summary
* Binary Classification : 비용 함수로 `BCE(Binary Cross Entropy)`를, 활성화 함수로 `sigmoid`를 사용
* Multilabel Classification : 비용 함수로 `CE(Cross Entropy)`를, 활성화 함수로 `softmax`를 사용